In [1]:
# predict gender
# tf-idf for creative_id
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse

train_merged=pd.read_csv('train_merged.csv')
predict_merged=pd.read_csv('predict_merged.csv')
total=pd.concat([train_merged,predict_merged],axis=0)
display(total)
v = TfidfVectorizer(lowercase=False,tokenizer=lambda x:x)

creative_id=total.groupby("user_id")["creative_id"].apply(list)
creative_id = v.fit_transform(creative_id)
advertiser_id=total.groupby("user_id")["advertiser_id"].apply(list)
advertiser_id = v.fit_transform(advertiser_id)
cr_ad=sparse.hstack((creative_id,advertiser_id),dtype=float)
display(cr_ad)
sparse.save_npz('cr_ad.npz', cr_ad)

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
# predict gender
%reset -f
# !pip install lightgbm
import lightgbm as lgb
import pandas as pd
from scipy import sparse
from sklearn.model_selection import KFold
import numpy as np

user_info=pd.read_csv("user_info.csv")
total=sparse.load_npz('cr_ad.npz')
X_train=total.tocsr()[:900000,:]
y_train=user_info["gender"]
X_test=total.tocsr()[900000:,:]
display(X_train,X_test)
predictions = np.zeros(X_test.shape[0])

folds = KFold(n_splits=5, shuffle=False)

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
    print("fold n°{}".format(fold_+1))
    display(trn_idx,val_idx)
    trn_data_X_train=X_train.tocsr()[trn_idx,:]
    val_data_X_train=X_train.tocsr()[val_idx,:]
    display(trn_data_X_train,val_data_X_train)
    trn_data = lgb.Dataset(trn_data_X_train, y_train.iloc[trn_idx])
    val_data = lgb.Dataset(val_data_X_train, y_train.iloc[val_idx])
    
    lgb_params = {'num_leaves': 2**6-1,
              'min_data_in_leaf': 25, 
              'objective':'binary',
              'max_depth': -1,
              'learning_rate': 0.1,
              'boosting': 'gbdt',
              'feature_fraction': 0.6,
              'bagging_fraction': 0.9,
              'bagging_seed': 11,
              'metric': 'auc',
              'seed':1024,
              'nthread':12,
             }
    model = lgb.train(lgb_params, 
                    trn_data,
                    num_boost_round=200, 
                    valid_sets = [trn_data, val_data], 
                    verbose_eval = 10, 
                    early_stopping_rounds = 100)
    predictions += model.predict(X_test, num_iteration=model.best_iteration) / folds.n_splits
    display(predictions)
    display(np.where(predictions>0.5))

display(predictions)

In [ ]:
submission=pd.DataFrame([i for i in range(3000001,4000001)],columns=["user_id"])
submission["predicted_age"]=[0 for _ in range(1000000)]
submission["predicted_gender"]=(predictions>0.5)*1
submission["predicted_gender"]+=1
display(submission)
submission.describe()
submission.to_csv("submission.csv",index=False)